# Project Part 3: Final Presentation

### CS-GY 6083 Principals of Database Systems - Spring 2024

* **Author**: [Nicola Maiorana]
* **Date**: [2024-03-23]
* **Email**: [nam10102@nyu.edu]
* **Class**: [CS-GY 6083]

## Overview of Presentation
- Project Overview
- IDE And Presentation Tools
- Code
    - ER Diagram
    - Tables DDL
    - Views DDL
    - Procedures DDL
    - Functions DDL
    - Triggers DDL

- Discussion
    - Normalization
    - Integrity Enforcement Used (Where/Why)
    - Isolation Level Used (Where/Why)
- Forms
    - Table Inserts
    - Table Deletes
    - Selects
- Reports

## Project Overview
For this project I will construct a database using MySQL to store information about record album: the name of the album, the recording date, the artists behind the album (bands), the members of the recording group, the record label which produced the album, genre and sales information. This will utilize a 2-tier architecture using Python as the primary Language.

I will construct business object classes to represent the data in Python using data classes. To map these data classes to the MySQL db, I will use the MySQL Connect coupled with dataclasses to create my Business Objects. Lastly I will construct a series of UI classes to perform Create, Read, Update and Delete operations on one of the tables.

The demonstration will consist of a Python stand-alone class connecting to the MySQL database and performing the CRUD operations on the database. The demonstration will be done using a Jupyter Notebook.

## IDE And Presentation Tools
- IDE: PyCharm Professional
- Presentation: Jupyter Notebook

## Code

### Import required modules

- tools.db_utils: Contains the database connection and query functions
- tools.mermaid_diagrams: Allows for the creation of Mermaid diagrams

In [2]:
import tools.db_utils as dbu
import tools.mermaid_diagrams as md

### ER Diagram

In [3]:
md.mm(md.er_diargram)

```sql
CREATE TABLE RECORD_ARTISTS (
    artist_id int NOT NULL AUTO_INCREMENT,
    artist_name varchar(255),
    PRIMARY KEY (artist_id)
);

CREATE TABLE GROUP_MEMBERS (
    member_id int NOT NULL AUTO_INCREMENT,
    member_name varchar(255),
    PRIMARY KEY (member_id)
);

CREATE TABLE MEMBERS_TO_ARTISTS (
    members_to_artists_id int NOT NULL AUTO_INCREMENT,
    member_id int,
    artist_id int,
    member_from_date date,
    member_to_date date,
    PRIMARY KEY (members_to_artists_id),
    FOREIGN KEY (member_id) REFERENCES GROUP_MEMBERS(member_id),
    FOREIGN KEY (artist_id) REFERENCES RECORD_ARTISTS(artist_id)
);

CREATE TABLE RECORD_GENRES (
    genre_id int NOT NULL AUTO_INCREMENT,
    genre_name varchar(255),
    genre_description varchar(255),
    PRIMARY KEY (genre_id),
    UNIQUE (genre_name)
);

CREATE TABLE RECORD_LABELS (
    record_label_id int NOT NULL AUTO_INCREMENT,
    record_label_name varchar(255),
    PRIMARY KEY (record_label_id),
    UNIQUE (record_label_name)
);

CREATE TABLE RECORD_ALBUMS (
    album_id int NOT NULL AUTO_INCREMENT,
    album_name varchar(255),
    release_date date,
    artist_id int,
    genre_id int,
    record_label_id int,
    PRIMARY KEY (album_id),
    FOREIGN KEY (artist_id) REFERENCES RECORD_ARTISTS(artist_id),
    FOREIGN KEY (genre_id) REFERENCES RECORD_GENRES(genre_id),
    FOREIGN KEY (record_label_id) REFERENCES RECORD_LABELS(record_label_id),
    UNIQUE (album_name, release_date)
);

CREATE TABLE RECORD_TRACKS (
    track_id int NOT NULL AUTO_INCREMENT,
    album_id int,
    track_name varchar(255),
    track_number int,
    genre_id int,
    PRIMARY KEY (track_id),
    FOREIGN KEY (album_id) REFERENCES RECORD_ALBUMS(album_id),
    FOREIGN KEY (genre_id) REFERENCES RECORD_GENRES(genre_id)
);

CREATE TABLE RECORD_SALES (
    sale_id int NOT NULL AUTO_INCREMENT,
    album_id int,
    sale_date date,
    sale_quantity int,
    unit_sale_price decimal,
    PRIMARY KEY (sale_id),
    FOREIGN KEY (album_id) REFERENCES RECORD_ALBUMS(album_id)
);
```

### Views DDL

#### Album Information View
```sql
CREATE VIEW album_information AS
    SELECT
        RECORD_ALBUMS.album_name,
        RECORD_ARTISTS.artist_name,
        RECORD_GENRES.genre_name,
        RECORD_LABELS.record_label_name,
        RECORD_ALBUMS.release_date
    FROM
        RECORD_ALBUMS
    JOIN RECORD_ARTISTS ON RECORD_ALBUMS.artist_id = RECORD_ARTISTS.artist_id
    JOIN RECORD_LABELS ON RECORD_ALBUMS.record_label_id = RECORD_LABELS.record_label_id
    JOIN RECORD_GENRES ON RECORD_ALBUMS.genre_id = RECORD_GENRES.genre_id;
```

In [4]:
pandas_index_settings = ['album_name']
display(dbu.sqlalchemy_query_to_df('select * from album_information order by release_date', pandas_index_settings))

,artist_name,genre_name,record_label_name,release_date
album_name,,,,
The Dark Side of the Moon,Pink Floyd,Rock,Harvest Capitol Records,1973-03-01
Boston,Boston,Rock,Epic Records,1976-08-25


#### Album Information Details View
```sql
CREATE VIEW album_information_details AS
    SELECT
        RECORD_ALBUMS.album_name,
        RECORD_ARTISTS.artist_name,
        record_genres.genre_name,
        RECORD_LABELS.record_label_name,
        RECORD_TRACKS.track_number,
        RECORD_TRACKS.track_name,
        RECORD_ALBUMS.release_date
    FROM
        RECORD_ALBUMS
    JOIN RECORD_ARTISTS ON RECORD_ALBUMS.artist_id = RECORD_ARTISTS.artist_id
    JOIN RECORD_LABELS ON RECORD_ALBUMS.record_label_id = RECORD_LABELS.record_label_id
    JOIN RECORD_TRACKS ON RECORD_ALBUMS.album_id = RECORD_TRACKS.album_id
    JOIN RECORD_GENRES ON RECORD_TRACKS.genre_id = RECORD_GENRES.genre_id
ORDER BY
    RECORD_ALBUMS.album_name, RECORD_TRACKS.track_number;
```

In [5]:
pandas_index_settings = ['album_name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
display(dbu.sqlalchemy_query_to_df('select * from album_information_details order by release_date', pandas_index_settings))

genre_name  \
album_name                release_date artist_name record_label_name       track_number              
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   
                                                                           9                  Rock   
                                                                           10                 Rock   
Boston                    1976-08-25   Boston      Epic Records            1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   

                                                                                                           track_name  
album_name                release_date artist_name record_label_name       track_number                                
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                              Speak to Me  
                                                                           2                     Breathe (In the Air)  
                                                                           3                               On the Run  
                                                                           4                                     Time  
                                                                           5                 The Great Gig in the Sky  
                                                                           6                                    Money  
                                                                           7                              Us and Them  
                                                                           8                      Any Colour You Like  
                                                                           9                             Brain Damage  
                                                                           10                                 Eclipse  
Boston                    1976-08-25   Boston      Epic Records            1                      More Than a Feeling  
                                                                           2                            Peace of Mind  
                                                                           3                       Foreplay/Long Time  
                                                                           4                         Rock & Roll Band  
                                                                           5                                   Smokin  
       

#### Band Members View
```sql
CREATE VIEW band_members AS
    SELECT
        RECORD_ARTISTS.artist_name,
        GROUP_MEMBERS.member_name,
        MEMBERS_TO_ARTISTS.member_from_date,
        MEMBERS_TO_ARTISTS.member_to_date
    FROM
        RECORD_ARTISTS
    JOIN MEMBERS_TO_ARTISTS ON RECORD_ARTISTS.artist_id = MEMBERS_TO_ARTISTS.artist_id
    JOIN GROUP_MEMBERS ON MEMBERS_TO_ARTISTS.member_id = GROUP_MEMBERS.member_id;
```

In [6]:
pandas_index_settings = ['artist_name', 'member_name']
display(dbu.sqlalchemy_query_to_df('select * from band_members order by artist_name, member_name', pandas_index_settings))

member_from_date member_to_date
artist_name member_name                                   
Boston      Barry Goudreau       1975-01-01     1981-01-01
            Brad Delp            1975-01-01     2007-01-01
            Fran Sheehan         1975-01-01     1986-01-01
            Sib Hashian          1975-01-01     1986-01-01
            Tom Scholz           1975-01-01     1989-01-01
Pink Floyd  David Gilmour        1968-01-01     1995-01-01
            Nick Mason           1965-01-01     1995-01-01
            Richard Wright       1965-01-01     1980-01-01
            Roger Waters         1965-01-01     1985-01-01
            Syd Barrett          1965-01-01     1968-01-01

### Procedures DDL


### Procedure to count the number of sales for a given album
```
DELIMITER //
CREATE PROCEDURE count_record_sales(IN album_id INT, OUT sales_count INT)
BEGIN
	SET @album_id = album_id;
    SELECT 
        COUNT(sale_id) INTO sales_count
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;
```


In [7]:
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        album_id = 1
        print(f'Number of sales for album 1 using select statement: {len(dbu.sqlalchemy_query_to_df("select * from record_sales where album_id = 1", "sale_id"))}')
        result = cur.callproc('count_record_sales', [album_id, 0])[1]
        print(f'Number of sales for album 1 using the stored procedure: {result}')

Number of sales for album 1 using select statement: 15
Number of sales for album 1 using the stored procedure: 15


### Procedure to get the total sales for all albums
```
DROP PROCEDURE IF EXISTS total_sales;
DELIMITER //
CREATE PROCEDURE total_record_sales(IN album_id INT, OUT total_sales NUMERIC)
BEGIN
	SET @album_id = album_id;
    SELECT 
        sum(sale_quantity * unit_sale_price) INTO total_sales
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;

In [8]:
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        album_id = 1
        print(f'Total sales for album 1 using select statement: {dbu.sqlalchemy_query_to_df("select sum(sale_quantity * unit_sale_price) as total_sales from record_sales where album_id = 1",)["total_sales"].values[0]}')
        result = float(cur.callproc('total_record_sales', (album_id, 0))[1])
        print(f'Total sales for album 1 using the stored procedure: {result}')

Total sales for album 1 using select statement: 150000000.0
Total sales for album 1 using the stored procedure: 150000000.0


## Functions DDL

```sql
DELIMITER //
CREATE FUNCTION album_summary (album_id int) RETURNS varchar(250)
	READS SQL DATA 
BEGIN
	DECLARE done int default false;
    DECLARE track_summary varchar(200);
	DECLARE album_summary varchar(50);
    DECLARE response_summary varchar(300);
    DECLARE album_name varchar(50);
    DECLARE release_date date;
    DECLARE track_number int;
    DECLARE track_name varchar(255);
	DECLARE result_cursor CURSOR FOR 
		SELECT 
			concat(" ", t.track_number, "-", t.track_name) AS track_summary
			FROM record_tracks t
			WHERE t.album_id = album_id
			ORDER BY t.track_number;
	DECLARE CONTINUE HANDLER FOR not found SET done = true;
	SET album_summary = "";
    SET response_summary = "";
	OPEN result_cursor;
	read_loop: LOOP 
		FETCH result_cursor INTO track_summary;
        IF done THEN
			LEAVE read_loop; 
		END IF;
        SET response_summary = CONCAT(response_summary, " ", track_summary);
    END LOOP;
    CLOSE result_cursor;
	SELECT 
		concat(r.album_name, " (", r.release_date, "): ") INTO album_summary
		FROM record_albums r
		WHERE r.album_id = album_id;
	SET response_summary = CONCAT(album_summary, response_summary);
    RETURN response_summary;
END //
DELIMITER ;
```

In [11]:
with dbu.get_connector() as conn:
    with conn.cursor() as cur:
        album_id = 1
        cur.execute(f'select album_summary(%s)', (album_id,))
        result = cur.fetchone()[0]
        print(f'Album summary:\n {result}')

Album summary:
 The Dark Side of the Moon (1973-03-01):   1-Speak to Me  2-Breathe (In the Air)  3-On the Run  4-Time  5-The Great Gig in the Sky  6-Money  7-Us and Them  8-Any Colour You Like  9-Brain Damage  10-Eclipse


## Trigger DDL

## Normalization

## Integrity Enforcement

## Isolation Level

## Forms
- Table Updates
- Table Deletes
- Selects

### Table Inserts

In [6]:
from project.business_objects.record_genres import RecordGenres

test_genre_name = 'TEST Genre'
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Cleanup
RecordGenres.delete_by_name(test_genre_name)

,genre_name,genre_description
genre_id,,
83,TEST Genre,A new genre


### Table Updates

In [19]:
from business_objects.record_genres import RecordGenres

test_genre_name = 'TEST Genre'
updated_genre_name = 'Updated Genre'
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
new_record.genre_name = updated_genre_name
new_record.genre_description = 'An updated genre'
print(f'Record to update: {new_record}')
updated_record = RecordGenres.update(new_record)
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{updated_genre_name}'",  'genre_id'))
# Cleanup
RecordGenres.delete_by_name(updated_genre_name)

,genre_name,genre_description
genre_id,,
21,TEST Genre,A new genre


Record to update: RecordGenres(genre_id=21, genre_name='Updated Genre', genre_description='An updated genre')


,genre_name,genre_description
genre_id,,
21,Updated Genre,An updated genre


### Table Deletes

In [23]:
from business_objects.record_genres import RecordGenres

test_genre_name = 'TEST Genre'
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
RecordGenres.delete(new_record.genre_id)
print(f'After delete by id: {new_record.genre_id}')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_id = '{new_record.genre_id}'",  'genre_id'))
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
RecordGenres.delete_by_name(new_record.genre_name)
print(f'After delete by name: {new_record.genre_name}')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))

,genre_name,genre_description
genre_id,,
35,TEST Genre,A new genre


After delete by id: 35


,genre_name,genre_description
genre_id,,


,genre_name,genre_description
genre_id,,
36,TEST Genre,A new genre


After delete by name: TEST Genre


,genre_name,genre_description
genre_id,,


### Table Reads

In [18]:
from business_objects.record_genres import RecordGenres
print(f'Read all genres')
for genre in RecordGenres.read_all()[:5]:
    print(genre)
    
print(f'Read by id: 5')
print(RecordGenres.read(5))
print(f'Read by name: Country')
print(RecordGenres.read_by_name('Country'))

Read all genres
RecordGenres(genre_id=1, genre_name='Rock', genre_description='Rock music')
RecordGenres(genre_id=2, genre_name='Pop', genre_description='Pop music')
RecordGenres(genre_id=3, genre_name='Rap', genre_description='Rap music')
RecordGenres(genre_id=4, genre_name='Country', genre_description='Country music')
RecordGenres(genre_id=5, genre_name='Jazz', genre_description='Jazz music')
Read by id: 5
RecordGenres(genre_id=5, genre_name='Jazz', genre_description='Jazz music')
Read by name: Country
[RecordGenres(genre_id=4, genre_name='Country', genre_description='Country music')]


## Reports